In [1]:
# Git Clone
! git clone https://github.com/roclark/sportsreference

fatal: destination path 'sportsreference' already exists and is not an empty directory.


In [2]:
cd sportsreference

/Users/tjsmith99/Portfolio/sportsAnalytics/2020PlayoffPrediction/data/sportsreference


In [3]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Import package and API
import pandas as pd
from sportsreference.nba.teams import Teams
from sportsreference.nba.schedule import Schedule

In [5]:
year = str(2020)
all_team_df = Teams(year = year).dataframes

In [6]:
#all_team_df['id'] = 
all_team_df['id'] = all_team_df['abbreviation'].apply(lambda x: x + " " + year)

In [7]:
all_team_df.shape

(30, 48)

In [8]:
for i in range(2019,1979,-1):
    print(i)
    year = str(i)
    temp_df = Teams(year = year).dataframes
    temp_df['id'] = temp_df['abbreviation'].apply(lambda x: x + " " + year)
    all_team_df = pd.concat([all_team_df, temp_df], axis = 0)

2019
2018
2017
2016
2015
2014
2013
2012
2011
2010
2009
2008
2007
2006
2005
2004
2003
2002
2001
2000
1999
1998
1997
1996
1995
1994
1993
1992
1991
1990
1989
1988
1987
1986
1985
1984
1983
1982
1981
1980


In [9]:
all_team_df['year'] = all_team_df['id'].apply(lambda x: x.split(' ')[1])

In [10]:
all_team_df.columns

Index(['abbreviation', 'assists', 'blocks', 'defensive_rebounds',
       'field_goal_attempts', 'field_goal_percentage', 'field_goals',
       'free_throw_attempts', 'free_throw_percentage', 'free_throws',
       'games_played', 'minutes_played', 'name', 'offensive_rebounds',
       'opp_assists', 'opp_blocks', 'opp_defensive_rebounds',
       'opp_field_goal_attempts', 'opp_field_goal_percentage',
       'opp_field_goals', 'opp_free_throw_attempts',
       'opp_free_throw_percentage', 'opp_free_throws',
       'opp_offensive_rebounds', 'opp_personal_fouls', 'opp_points',
       'opp_steals', 'opp_three_point_field_goal_attempts',
       'opp_three_point_field_goal_percentage', 'opp_three_point_field_goals',
       'opp_total_rebounds', 'opp_turnovers',
       'opp_two_point_field_goal_attempts',
       'opp_two_point_field_goal_percentage', 'opp_two_point_field_goals',
       'personal_fouls', 'points', 'rank', 'steals',
       'three_point_field_goal_attempts', 'three_point_field_goa

In [11]:
def get_win_rate(tid):
    abb = tid.split(' ')[0]
    year = int(tid.split(' ')[1])
    games = 0
    wins = 0
    playoff_games = 0
    playoff_wins = 0
    
    for game in Schedule(abb, year = year):
        if game.playoffs == False:
            games += 1
            if game.result == 'Win':
                wins += 1
        elif game.playoffs == True:
            playoff_games += 1
            if game.result == 'Win':
                playoff_wins += 1
            
    return wins, games, playoff_wins, playoff_games

In [12]:
get_win_rate('DAL 2020')

(48, 75, 0, 0)

In [13]:
total_wins_dict = {}

In [14]:
for y in range(2019, 1979, -1):
    
    tids = all_team_df[all_team_df['year'] == str(y)]['id']
    
    wins_dict = {'id': [], 'wins': [], 'games': [], 'playoff_wins': [], 'playoff_games': []}

    for i, tid in enumerate(tids):
        wins, games, playoff_wins, playoff_games = get_win_rate(tid)
        wins_dict['id'].append(tid)
        wins_dict['wins'].append(wins)
        wins_dict['games'].append(games)
        wins_dict['playoff_wins'].append(playoff_wins)
        wins_dict['playoff_games'].append(playoff_games)
        
    total_wins_dict[y] = wins_dict
    print(y)

2019
2018
2017
2016
2015
2014
2013
2012
2011
2010
2009
2008
2007
2006
2005
2004
2003
2002
2001
2000
1999
1998
1997
1996
1995
1994
1993
1992
1991
1990
1989
1988
1987
1986
1985
1984
1983
1982
1981
1980


In [16]:
total_wins_df = pd.DataFrame(total_wins_dict[2019])

for key in list(total_wins_dict.keys())[1:]:
    temp_df = pd.DataFrame(total_wins_dict[key])
    total_wins_df = pd.concat([total_wins_df, temp_df], axis = 0)

all_data = pd.merge(left = all_team_df, right = total_wins_df, how = 'inner', left_on = 'id', right_on = 'id')
all_data.head()

,abbreviation,assists,blocks,defensive_rebounds,field_goal_attempts,field_goal_percentage,field_goals,free_throw_attempts,free_throw_percentage,free_throws,...,turnovers,two_point_field_goal_attempts,two_point_field_goal_percentage,two_point_field_goals,id,year,wins,games,playoff_wins,playoff_games
0,MIL,2136,486,3316,7471,0.476,3555,1904,0.773,1471,...,1137,4337,0.565,2450,MIL 2019,2019,60,82,10,15
1,GSW,2413,525,2990,7361,0.491,3612,1672,0.801,1339,...,1169,4537,0.557,2525,GSW 2019,2019,57,82,14,22
2,NOP,2216,441,2969,7563,0.473,3581,1921,0.761,1462,...,1215,5114,0.536,2739,NOP 2019,2019,33,82,0,0
3,PHI,2207,432,3025,7233,0.471,3407,2258,0.771,1742,...,1223,4759,0.529,2518,PHI 2019,2019,51,82,7,12
4,LAC,1970,385,2936,7178,0.471,3384,2340,0.792,1853,...,1193,5060,0.507,2563,LAC 2019,2019,48,82,2,6


In [29]:
all_data.to_csv('all_data1980-2019.csv', sep='\t')

In [31]:
# Build 2020 dataset
for y in range(2020, 2019, -1):
    
    tids = all_team_df[all_team_df['year'] == str(y)]['id']
    
    wins_dict = {'id': [], 'wins': [], 'games': [], 'playoff_wins': [], 'playoff_games': []}

    for i, tid in enumerate(tids):
        wins, games, playoff_wins, playoff_games = get_win_rate(tid)
        wins_dict['id'].append(tid)
        wins_dict['wins'].append(wins)
        wins_dict['games'].append(games)
        wins_dict['playoff_wins'].append(playoff_wins)
        wins_dict['playoff_games'].append(playoff_games)
        
    total_wins_dict[y] = wins_dict
    print(y)

2020


In [35]:
total_wins_df = pd.DataFrame(total_wins_dict[2020])

for key in list(total_wins_dict.keys())[1:]:
    temp_df = pd.DataFrame(total_wins_dict[key])
    total_wins_df = pd.concat([total_wins_df, temp_df], axis = 0)

_2020data = pd.merge(left = all_team_df, right = total_wins_df, how = 'inner', left_on = 'id', right_on = 'id')
_2020data.head()
_2020data.to_csv('data2020.csv',sep='\t')

## Works cited
For building of dataframe with wins
- https://towardsdatascience.com/beating-the-odds-8d26b1a83f1b
- https://github.com/alexmuhr/Over_Under_Achiever_Analysis/tree/fbe575df5ec397679bc10d1cfe5b0b09eb30f578

Sportsreference Documentation
- https://sportsreference.readthedocs.io/en/stable/nba.html#teams